# This Notebook is used to find the static points during an experiment with the three prisms on the robot.

In [ ]:
import scipy.linalg
import scipy.optimize
import sys
import numpy as np
import importlib
import matplotlib.pyplot as plt
import scripts.theodolite_function as tfu
tfu = importlib.reload(tfu)
from scipy.interpolate import splprep, splev
from scipy import interpolate
import seaborn as sns
from matplotlib.colors import BoundaryNorm
from matplotlib.ticker import MaxNLocator
from scipy.spatial.transform import Rotation as R
import time
from tqdm import tqdm
import scripts.theodolite_utils as tu
import scripts.resection_functions as rf
from scipy import stats


In [ ]:
# 17 02 2022 / 24 02 2022
Dist_prism_12_240222 = 0.8882613537922258
Dist_prism_13_240222 = 0.8262903370535766
Dist_prism_23_240222 = 1.0377436460567973
Dist_GPS_12_240222 = 0.7809786915630756
Dist_GPS_13_240222 = 0.8378323585433202
Dist_GPS_23_240222 = 0.5191788936274444   

# 07 03 2022
Dist_prism_12_070322 = 0.8832836583456264
Dist_prism_13_070322 = 0.8247449387008166
Dist_prism_23_070322 = 1.039409793600046
Dist_GPS_12_070322 = 0.7781099344563802
Dist_GPS_13_070322 = 0.8431026980109139
Dist_GPS_23_070322 = 0.5157221841320069

# 12 03 2022
Dist_prism_12_120322 = 0.8860734460339026
Dist_prism_13_120322 = 0.8303896290142598
Dist_prism_23_120322 = 1.0393117663247973
Dist_GPS_12_120322 = 0.7829424296161074
Dist_GPS_13_120322 = 0.8415176433032493
Dist_GPS_23_120322 = 0.5161917038941966

# 14 03 2022 / 16 03 2022
Dist_prism_12_140322 = 0.8873171656784946
Dist_prism_13_140322 = 0.8272212117473343
Dist_prism_23_140322 = 1.0379270641796363
Dist_GPS_12_140322 = 0.7829081036179948
Dist_GPS_13_140322 = 0.8424633975958637
Dist_GPS_23_140322 = 0.5165807364575292

# 31 03 2022
Dist_prism_12_310322 = 0.8873171656784946
Dist_prism_13_310322 = 0.8272212117473343
Dist_prism_23_310322 = 1.0379270641796363
Dist_GPS_12_310322 = 0.7829081036179948
Dist_GPS_13_310322 = 0.8424633975958637
Dist_GPS_23_310322 = 0.5165807364575292

# 27 04 2022
Dist_prism_12_270422 = 0.8856608085851714
Dist_prism_13_270422 = 0.8264452483880412
Dist_prism_23_270422 = 1.0384219210289538
Dist_GPS_12_270422 = 0.7809547832390561
Dist_GPS_13_270422 = 0.8259590286006459
Dist_GPS_23_270422 = 0.5191274645630007  

# 27 04 2022 soir
Dist_prism_12_270422_soir = 0.8868846942534616
Dist_prism_13_270422_soir = 0.829169129877633
Dist_prism_23_270422_soir = 1.0387571594276301
Dist_GPS_12_270422_soir = 0.7837749633840095
Dist_GPS_13_270422_soir = 0.8323125081688734
Dist_GPS_23_270422_soir = 0.5187055120945835

# 05 05 2022
Dist_prism_12_050522 = 0.3819811991689936
Dist_prism_13_050522 = 0.4426382054042266
Dist_prism_23_050522 = 0.2564685508415531

# 13 05 2022
Dist_prism_12_130522 = 0.8113569618671205
Dist_prism_13_130522 = 0.8695313906832193
Dist_prism_23_130522 = 1.0383292631343506
Dist_GPS_12_130522 = 0.7800801192552722
Dist_GPS_13_130522 = 0.8396027021467387
Dist_GPS_23_130522 = 0.5157656903777014

# 23 05 2022
Dist_prism_12_230522 = 0.3851913749758221
Dist_prism_13_230522 = 0.4433899497583272
Dist_prism_23_230522 = 0.25861327466684897

# 25 05 2022
Dist_prism_12_250522 = 0.9095421527752512
Dist_prism_13_250522 = 0.7356677023921305
Dist_prism_23_250522 = 1.0283049452358466

# 22 06 2022
Dist_prism_12_220622 = 0.7359204697527427
Dist_prism_13_220622 = 0.9055056357941131
Dist_prism_23_220622 = 1.0268390664025775

# 30 06 2022
Dist_prism_12_300622 = 0.7297583705742492
Dist_prism_13_300622 = 0.9060320633355149
Dist_prism_23_300622 = 1.0319114860615324

# 11 07 2022
Dist_prism_12_110722 = 0.7290056530579362
Dist_prism_13_110722 = 0.906995998154099
Dist_prism_23_110722 = 1.029264651249632

# 17 07 2022
Dist_prism_12_170722 = 0.39002864329744197 
Dist_prism_13_170722 = 0.4452628757931205 
Dist_prism_23_170722 = 0.2597453130254009

linear_inter_prism_file_name = ['./data/prediction/TS/20220224/filtered/f-2-1-1-1-6-1-L_',    # bad
                                './data/prediction/TS/20220307/filtered/f-2-1-1-1-6-1-L_',    # okay
                                './data/prediction/TS/20220312/filtered/f-2-1-1-1-6-1-L_',    # bad!
                                './data/prediction/TS/20220314/filtered/f-2-1-1-1-6-1-L_',    # okay
                                './data/prediction/TS/20220316/filtered/f-2-1-1-1-6-1-L_',    # not great
                                './data/prediction/TS/20220331-1/filtered/f-2-1-1-1-6-1-L_',  # okay
                                './data/prediction/TS/20220331-2/filtered/f-2-1-1-1-6-1-L_',  # bad
                                './data/prediction/TS/20220427/filtered/f-2-1-1-1-6-1-L_',    # not great
                                './data/prediction/TS/20220505_cones/filtered/f-2-1-1-1-6-1-L_',  # badder
                                './data/prediction/TS/20220505_empty/filtered/f-2-1-1-1-6-1-L_',  # badder
                                './data/prediction/TS/20220513/01/filtered/f-2-1-1-1-6-1-L_',   # not bad!
                                './data/prediction/TS/20220513/02/filtered/f-2-1-1-1-6-1-L_',   # badder
                                './data/prediction/TS/20220513/03/filtered/f-2-1-1-1-6-1-L_',   # not great
                                './data/prediction/TS/20220513/04/filtered/f-2-1-1-1-6-1-L_',   # best one yet
                                #'./data/prediction/TS/20220513/05/filtered/f-2-1-1-1-6-1-L_',   # extremely horrible
                                './data/prediction/TS/20220513/06/filtered/f-2-1-1-1-6-1-L_',   # okay
                                './data/prediction/TS/20220523_cones/filtered/f-2-1-1-1-6-1-L_',
                                './data/prediction/TS/20220523_constrained/filtered/f-2-1-1-1-6-1-L_',
                                './data/prediction/TS/20220523_empty/filtered/f-2-1-1-1-6-1-L_',
                                './data/prediction/TS/20220523_other_tunnel/filtered/f-2-1-1-1-6-1-L_',
                                './data/prediction/TS/20220525/01/filtered/f-2-1-1-1-6-1-L_',
                                './data/prediction/TS/20220525/02/filtered/f-2-1-1-1-6-1-L_',
                                './data/prediction/TS/20220622-1/filtered/f-2-1-1-1-6-1-L_',
                                './data/prediction/TS/20220622-2//filtered/f-2-1-1-1-6-1-L_',
                                './data/prediction/TS/20220630-1/filtered/f-2-1-1-1-6-1-L_',
                                './data/prediction/TS/20220630-2/filtered/f-2-1-1-1-6-1-L_',    # okay
                                './data/prediction/TS/20220711/02/filtered/f-2-1-1-1-6-1-L_',   # bad
                                './data/prediction/TS/20220717/02/filtered/f-2-1-1-1-6-1-L_',
                                './data/prediction/TS/20220717/03/filtered/f-2-1-1-1-6-1-L_',
                                './data/prediction/TS/20220717/04/filtered/f-2-1-1-1-6-1-L_'
                               ]
file_name_marker = ['./data/total_station/20220224/20220224_theodolite_reference_prisms.txt',
                    './data/total_station/20220307/theodolite_reference_prisms.txt',
                    './data/total_station/20220312/theodolite_reference_prisms.txt',
                    './data/total_station/20220314/theodolite_reference_prisms.txt',
                    './data/total_station/20220316/theodolite_reference_prisms.txt',
                    './data/total_station/20220331-1/theodolite_reference_prisms.txt',
                    './data/total_station/20220331-2/theodolite_reference_prisms.txt',
                    './data/total_station/20220427/theodolite_reference_prisms.txt',
                    './data/total_station/20220505_cones/theodolite_reference_prisms.txt',
                    './data/total_station/20220505_empty/theodolite_reference_prisms.txt',
                    './data/total_station/20220513_morning_lidar/theodolite_reference_prisms.txt',
                    './data/total_station/20220513_morning_without_lidar_1/theodolite_reference_prisms.txt',
                    './data/total_station/20220513_morning_without_lidar_2/theodolite_reference_prisms.txt',
                    './data/total_station/20220513_evening_lidar/theodolite_reference_prisms.txt',
                    #'./data/total_station/20220513_evening_without_lidar_1/theodolite_reference_prisms.txt',
                    './data/total_station/20220513_evening_without_lidar_2/theodolite_reference_prisms.txt',
                    './data/total_station/20220523_cones/theodolite_reference_prisms.txt',
                    './data/total_station/20220523_constrained/theodolite_reference_prisms.txt',
                    './data/total_station/20220523_empty/theodolite_reference_prisms.txt',
                    './data/total_station/20220523_other_tunnel/theodolite_reference_prisms.txt',
                    './data/total_station/20220525/theodolite_reference_prisms.txt',
                    './data/total_station/20220525/theodolite_reference_prisms.txt',
                    './data/total_station/20220622-1/theodolite_reference_prisms_2.txt',
                    './data/total_station/20220622-2/theodolite_reference_prisms_2.txt',
                    './data/total_station/20220630-1/theodolite_reference_prisms_without_first_two.txt',
                    './data/total_station/20220630-2/theodolite_reference_prisms_without_first_two.txt',
                    './data/total_station/20220711/theodolite_reference_prisms_all.txt',
                    './data/total_station/20220717/theodolite_reference_prisms.txt',
                    './data/total_station/20220717/theodolite_reference_prisms.txt',
                    './data/total_station/20220717/theodolite_reference_prisms.txt'
                   ]
                    

Inter_distance = [[Dist_prism_12_240222,Dist_prism_13_240222,Dist_prism_23_240222,Dist_GPS_12_240222,Dist_GPS_13_240222,Dist_GPS_23_240222],
                  [Dist_prism_12_070322,Dist_prism_13_070322,Dist_prism_23_070322,Dist_GPS_12_070322,Dist_GPS_13_070322,Dist_GPS_23_070322],
                  [Dist_prism_12_120322,Dist_prism_13_120322,Dist_prism_23_120322,Dist_GPS_12_120322,Dist_GPS_13_120322,Dist_GPS_23_120322],
                  [Dist_prism_12_140322,Dist_prism_13_140322,Dist_prism_23_140322,Dist_GPS_12_140322,Dist_GPS_13_140322,Dist_GPS_23_140322],
                  [Dist_prism_12_140322,Dist_prism_13_140322,Dist_prism_23_140322,Dist_GPS_12_140322,Dist_GPS_13_140322,Dist_GPS_23_140322],
                  [Dist_prism_12_310322,Dist_prism_13_310322,Dist_prism_23_310322,Dist_GPS_12_310322,Dist_GPS_13_310322,Dist_GPS_23_310322],
                  [Dist_prism_12_310322,Dist_prism_13_310322,Dist_prism_23_310322,Dist_GPS_12_310322,Dist_GPS_13_310322,Dist_GPS_23_310322],
                  [Dist_prism_12_270422,Dist_prism_13_270422,Dist_prism_23_270422,Dist_GPS_12_270422,Dist_GPS_13_270422,Dist_GPS_23_270422],
                  [Dist_prism_12_050522,Dist_prism_13_050522,Dist_prism_23_050522],
                  [Dist_prism_12_050522,Dist_prism_13_050522,Dist_prism_23_050522],
                  [Dist_prism_12_130522,Dist_prism_13_130522,Dist_prism_23_130522,Dist_GPS_12_130522,Dist_GPS_13_130522,Dist_GPS_23_130522],
                  [Dist_prism_12_130522,Dist_prism_13_130522,Dist_prism_23_130522,Dist_GPS_12_130522,Dist_GPS_13_130522,Dist_GPS_23_130522],
                  [Dist_prism_12_130522,Dist_prism_13_130522,Dist_prism_23_130522,Dist_GPS_12_130522,Dist_GPS_13_130522,Dist_GPS_23_130522],
                  [Dist_prism_12_130522,Dist_prism_13_130522,Dist_prism_23_130522,Dist_GPS_12_130522,Dist_GPS_13_130522,Dist_GPS_23_130522],
                  #[Dist_prism_12_130522,Dist_prism_13_130522,Dist_prism_23_130522,Dist_GPS_12_130522,Dist_GPS_13_130522,Dist_GPS_23_130522],
                  [Dist_prism_12_130522,Dist_prism_13_130522,Dist_prism_23_130522,Dist_GPS_12_130522,Dist_GPS_13_130522,Dist_GPS_23_130522],               
                  [Dist_prism_12_230522,Dist_prism_13_230522,Dist_prism_23_230522],
                  [Dist_prism_12_230522,Dist_prism_13_230522,Dist_prism_23_230522],
                  [Dist_prism_12_230522,Dist_prism_13_230522,Dist_prism_23_230522],
                  [Dist_prism_12_230522,Dist_prism_13_230522,Dist_prism_23_230522],
                  [Dist_prism_12_250522,Dist_prism_13_250522,Dist_prism_23_250522],
                  [Dist_prism_12_250522,Dist_prism_13_250522,Dist_prism_23_250522],
                  [Dist_prism_12_220622,Dist_prism_13_220622,Dist_prism_23_220622],
                  [Dist_prism_12_220622,Dist_prism_13_220622,Dist_prism_23_220622],
                  [Dist_prism_12_300622,Dist_prism_13_300622,Dist_prism_23_300622],
                  [Dist_prism_12_300622,Dist_prism_13_300622,Dist_prism_23_300622],
                  [Dist_prism_12_110722,Dist_prism_13_110722,Dist_prism_23_110722],
                  [Dist_prism_12_170722,Dist_prism_13_170722,Dist_prism_23_170722],
                  [Dist_prism_12_170722,Dist_prism_13_170722,Dist_prism_23_170722],
                  [Dist_prism_12_170722,Dist_prism_13_170722,Dist_prism_23_170722]
                 ]

# La section suivante filtre les datasets pour conserver seulement les points statiques

In [ ]:
path_to_save = ['./data/static_points_during_experiment/20220224/f-2-1-1-1-6-1-L_',
                './data/static_points_during_experiment/20220307/f-2-1-1-1-6-1-L_',
                './data/static_points_during_experiment/20220312/f-2-1-1-1-6-1-L_',
                './data/static_points_during_experiment/20220314/f-2-1-1-1-6-1-L_',
                './data/static_points_during_experiment/20220316/f-2-1-1-1-6-1-L_',
                './data/static_points_during_experiment/20220331-1/f-2-1-1-1-6-1-L_',
                './data/static_points_during_experiment/20220331-2/f-2-1-1-1-6-1-L_',
                './data/static_points_during_experiment/20220427/f-2-1-1-1-6-1-L_',
                './data/static_points_during_experiment/20220505_cones/f-2-1-1-1-6-1-L_',
                './data/static_points_during_experiment/20220505_empty/f-2-1-1-1-6-1-L_',
                './data/static_points_during_experiment/20220513/01/f-2-1-1-1-6-1-L_',
                './data/static_points_during_experiment/20220513/02/f-2-1-1-1-6-1-L_',
                './data/static_points_during_experiment/20220513/03/f-2-1-1-1-6-1-L_',
                './data/static_points_during_experiment/20220513/04/f-2-1-1-1-6-1-L_',
                './data/static_points_during_experiment/20220513/06/f-2-1-1-1-6-1-L_',
                './data/static_points_during_experiment/20220523_cones/f-2-1-1-1-6-1-L_',
                './data/static_points_during_experiment/20220523_constrained/f-2-1-1-1-6-1-L_',
                './data/static_points_during_experiment/20220523_empty/f-2-1-1-1-6-1-L_',
                './data/static_points_during_experiment/20220523_other_tunnel/f-2-1-1-1-6-1-L_',
                './data/static_points_during_experiment/20220525/01/f-2-1-1-1-6-1-L_',
                './data/static_points_during_experiment/20220525/02/f-2-1-1-1-6-1-L_',
                './data/static_points_during_experiment/20220622-1/f-2-1-1-1-6-1-L_',
                './data/static_points_during_experiment/20220622-2/f-2-1-1-1-6-1-L_',
                './data/static_points_during_experiment/20220630-1/f-2-1-1-1-6-1-L_',
                './data/static_points_during_experiment/20220630-2/f-2-1-1-1-6-1-L_',
                './data/static_points_during_experiment/20220711/02/f-2-1-1-1-6-1-L_',
                './data/static_points_during_experiment/20220717/02/f-2-1-1-1-6-1-L_',
                './data/static_points_during_experiment/20220717/03/f-2-1-1-1-6-1-L_',
                './data/static_points_during_experiment/20220717/04/f-2-1-1-1-6-1-L_'
]

rate = 10

# Adjust the speed limiter to vary what we consider static or not
speed_limiter = 0.01

for i,j in zip(linear_inter_prism_file_name, path_to_save):
    trimble_1 = tu.read_prediction_data_resection_csv_file(i + "1.csv")
    trimble_2 = tu.read_prediction_data_resection_csv_file(i + "2.csv")
    trimble_3 = tu.read_prediction_data_resection_csv_file(i + "3.csv")
    
    index1 = tu.find_not_moving_points_GP(np.array(trimble_1), speed_limiter, 1/rate)

    p1 = np.array(trimble_1)[index1, :5]
    p2 = np.array(trimble_2)[index1, :5]
    p3 = np.array(trimble_3)[index1, :5]
    np.savetxt(j+"1.csv", p1, delimiter=" ")
    np.savetxt(j+"2.csv", p2, delimiter=" ")
    np.savetxt(j+"3.csv", p3, delimiter=" ")


# La cellule suivante itère sur les données afin de déterminer le nombre de points statiques différents au cours de chaque expérience

In [ ]:
nb_static_points = np.zeros(shape=len(path_to_save))
index_start = []
index_end = []
counter = 0
for i in path_to_save:
    trimble_1 = tu.read_prediction_data_resection_csv_file(i + "1.csv")
    start_index = []
    end_index = []
    
    if len(trimble_1) > 0:
        nb_static_points[counter] += 1
        start_index.append(0)

        for j in range(len(trimble_1)-1):

            if np.linalg.norm(trimble_1[j][1:5]-trimble_1[j+1][1:5]) > 0.1:
                nb_static_points[counter] += 1
                end_index.append(j)
                start_index.append(j+1)

        if len(start_index) != len(end_index):
            end_index.append(len(trimble_1)-1)
        
    print("For ", i, "the number of static points is: ", nb_static_points[counter])
    index_start.append(start_index)
    index_end.append(end_index)
    counter += 1
# print(index_start)
# print(index_end)

In [ ]:
static_poses_1 = []
static_poses_2 = []
static_poses_3 = []

for i, j, k in zip(path_to_save, index_start, index_end):
    # print(i, '\n')
    # print(j[0], '\n')
    # print(k[0], '\n')
    trimble_1 = tu.read_prediction_data_resection_csv_file(i + "1.csv")
    trimble_2 = tu.read_prediction_data_resection_csv_file(i + "2.csv")
    trimble_3 = tu.read_prediction_data_resection_csv_file(i + "3.csv")
    # print(trimble_1[j[0]:k[0]], '\n')
    pose_1 = []
    pose_2 = []
    pose_3 = []
    for start,end in zip(j,k):
        mean_1 = np.mean(trimble_1[start:end], axis=0)
        mean_2 = np.mean(trimble_2[start:end], axis=0)
        mean_3 = np.mean(trimble_3[start:end], axis=0)
        pose_1.append([mean_1[1], mean_1[2], mean_1[3], mean_1[4]])
        pose_2.append([mean_2[1], mean_2[2], mean_2[3], mean_2[4]])
        pose_3.append([mean_3[1], mean_3[2], mean_3[3], mean_3[4]])
    static_poses_1.append(pose_1)
    static_poses_2.append(pose_2)
    static_poses_3.append(pose_3)

# print(static_poses_1, '\n')
# print(static_poses_2, '\n')
# print(static_poses_3, '\n')

In [ ]:
i = 0
print(static_poses_1[i])
for j in range(len(static_poses_1[i])):
    plt.scatter([static_poses_1[i][j][0], static_poses_2[i][j][0], static_poses_3[i][j][0]], [static_poses_1[i][j][1], static_poses_2[i][j][1], static_poses_3[i][j][1]])

In [ ]:
static_cp_1 = []
static_cp_2 = []
static_cp_3 = []
TF1 = []
TF2 = []
TF3 = []

for i in file_name_marker:
    ts1_static, ts2_static, ts3_static, T1_static, T12_static, T13_static = tu.read_marker_file(
            file_name=i, theodolite_reference_frame=1)
    static_cp_1.append(T1_static@ts1_static)
    static_cp_2.append(T12_static@ts2_static)
    static_cp_3.append(T13_static@ts3_static)
    TF1.append(T1_static)
    TF2.append(T12_static)
    TF3.append(T13_static)

In [ ]:
i = 0
for j in range(len(static_cp_1[i][0])):
    plt.scatter([static_cp_1[i][0][j], static_cp_2[i][0][j], static_cp_3[i][0][j]], [static_cp_1[i][1][j], static_cp_2[i][1][j], static_cp_3[i][1][j]])


# This next cell plots the static points and the control points together

In [ ]:
for i in range(len(static_poses_1)):
    static_poses_1[i] = np.array(static_poses_1[i]).T
    static_poses_2[i] = np.array(static_poses_2[i]).T
    static_poses_3[i] = np.array(static_poses_3[i]).T
    
    static_poses_1[i] = T1_static@static_poses_1[i]
    static_poses_2[i] = T12_static@static_poses_2[i]
    static_poses_3[i] = T13_static@static_poses_3[i]

In [ ]:
i = 3
for j in range(len(static_cp_1[i][0])):
    plt.scatter([static_cp_1[i][0][j], static_cp_2[i][0][j], static_cp_3[i][0][j]], [static_cp_1[i][1][j], static_cp_2[i][1][j], static_cp_3[i][1][j]], color='b')
    
for j in range(len(static_poses_1[i][0])):
    plt.scatter([static_poses_1[i][0][j], static_poses_2[i][0][j], static_poses_3[i][0][j]], [static_poses_1[i][1][j], static_poses_2[i][1][j], static_poses_3[i][1][j]], color='r')
print('Static poses: ', len(static_poses_1[i][0]))
print('Control points: ', len(static_cp_1[i][0]))